In [2]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
import torch
import torch.nn as nn
import os
import torchvision as tv
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import time
import copy
from torch.optim import Adam
plt.ion()  

In [3]:
use_gpu = 0
if use_gpu:
    torch.cuda.set_device(CUDA_DEVICE)

count=0

In [4]:
data_transform = {
    'Train' : tv.transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        #transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ]),
    'Val' : tv.transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ]),
    'Test' : tv.transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])
        
}

In [5]:
data_folder = "../Project/dataset"
datasets = ['Train','Val','Test']
image_data = {}
for x in datasets:
    image_data[x] = tv.datasets.ImageFolder(data_folder+'/'+x, transform=data_transform[x])
    
dataset_sizes = {x: len(image_data[x]) for x in datasets}

In [6]:
data_loader = {}
for x in datasets:
    data_loader[x] = torch.utils.data.DataLoader(image_data[x], batch_size=32,
        shuffle=True, num_workers=4
    )
class_names = image_data['Train'].classes

In [7]:
dset_sizes = {'Train' : 2900, 'Val' : 725, 'Test' : 28}

In [8]:
def train_model(model, criterion, optimizer, lr_scheduler, num_epochs=100):
    since = time.time()

    best_model = model
    best_acc = 0.0
    store_train_loss = []
    store_train_acc = []
    store_val_loss = []
    store_val_acc = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['Train', 'Val']:
            if phase == 'train':
                mode='train'
                optimizer = lr_scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
            else:
                model.eval()
                mode='val'

            running_loss = 0.0
            running_corrects = 0

            counter=0
            # Iterate over data.
            for data in data_loader[phase]:
                inputs, labels = data
                #print(inputs.size())
                # wrap them in Variable
                if use_gpu:
                    try:
                        inputs, labels = Variable(inputs.float().cuda()),                             
                        Variable(labels.long().cuda())
                    except:
                        print(inputs,labels)
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # Set gradient to zero to delete history of computations in previous epoch. Track operations so that differentiation can be done automatically.
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                
                loss = criterion(outputs, labels)
                # print('loss done')                
                # Just so that you can keep track that something's happening and don't feel like the program isn't running.
                # if counter%10==0:
                #     print("Reached iteration ",counter)
                counter+=1

                # backward + optimize only if in training phase
                if phase == 'Train':
                    # print('loss backward')
                    loss.backward()
                    # print('done loss backward')
                    optimizer.step()
                    # print('done optim')
                # print evaluation statistics
                try:
                    # running_loss += loss.data[0]
                    running_loss += loss.item()
                    # print(labels.data)
                    # print(preds)
                    running_corrects += torch.sum(preds == labels.data)
                    # print('running correct =',running_corrects)
                except:
                    print('unexpected error, could not calculate loss or do a sum.')
            print('trying epoch loss')
            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects.item() / float(dset_sizes[phase])
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            if phase == 'Train':
                store_train_loss.append(epoch_loss)
                store_train_acc.append(epoch_acc)
            elif phase == 'Val':
                store_val_loss.append(epoch_loss)
                store_val_acc.append(epoch_acc)


            # deep copy the model
            if phase == 'Val':
                #if USE_TENSORBOARD:
                #    foo.add_scalar_value('epoch_loss',epoch_loss,step=epoch)
                #    foo.add_scalar_value('epoch_acc',epoch_acc,step=epoch)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)
                    print('new best accuracy = ',best_acc)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('returning and looping back')
    return best_model

# This function changes the learning rate over the training model.
def exp_lr_scheduler(optimizer, epoch, init_lr=0.01, lr_decay_epoch=1):
    """Decay learning rate by a factor of DECAY_WEIGHT every lr_decay_epoch epochs."""
    lr = init_lr * (1**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

model_ft = models.alexnet(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
num_features = model_ft.classifier[6].in_features
features = list(model_ft.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
model_ft.classifier = nn.Sequential(*features) 


criterion = nn.CrossEntropyLoss()

if use_gpu:
    criterion.cuda()
    model_ft.cuda()

optimizer_ft = optim.RMSprop(model_ft.parameters(), lr=0.0001)



# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

# Save model
#model_ft.state_dict('fine_tuned_best_model.pt')
#model_ft.state_dict('mytraining.pt')


Epoch 0/9
----------
trying epoch loss
Train Loss: 0.0881 Acc: 0.2352
trying epoch loss
Val Loss: 0.0698 Acc: 0.4538
new best accuracy =  0.45379310344827584
Epoch 1/9
----------
trying epoch loss
Train Loss: 0.0749 Acc: 0.3721
trying epoch loss
Val Loss: 0.0587 Acc: 0.5848
new best accuracy =  0.5848275862068966
Epoch 2/9
----------
trying epoch loss
Train Loss: 0.0679 Acc: 0.4483
trying epoch loss
Val Loss: 0.0525 Acc: 0.6193
new best accuracy =  0.6193103448275862
Epoch 3/9
----------
trying epoch loss
Train Loss: 0.0652 Acc: 0.4731
trying epoch loss
Val Loss: 0.0481 Acc: 0.6428
new best accuracy =  0.6427586206896552
Epoch 4/9
----------
trying epoch loss
Train Loss: 0.0604 Acc: 0.5076
trying epoch loss
Val Loss: 0.0441 Acc: 0.7021
new best accuracy =  0.7020689655172414
Epoch 5/9
----------
trying epoch loss
Train Loss: 0.0589 Acc: 0.5128
trying epoch loss
Val Loss: 0.0419 Acc: 0.7172
new best accuracy =  0.7172413793103448
Epoch 6/9
----------
trying epoch loss
Train Loss: 0.0564

In [9]:
model_ft.eval()
torch.save(model_ft.state_dict(), 'mytraining3.pt') 
#print(model_ft.state_dict())

In [10]:
def test(m):
    print(type(m))
    test_acc = 0.0
    for i, (images, labels) in enumerate(data_loader['Test']):

#         if cuda_avail:
#             images = Variable(images.cuda())
#             labels = Variable(labels.cuda())

        # Predict classes using images from the test set
        outputs = m(images)
        _, prediction = torch.max(outputs.data, 1)
        
        test_acc += torch.sum(prediction == labels.data)

    # Compute the average acc and loss over all 10000 test images
    test_acc = test_acc.item() / dset_sizes['Test']
    print(dset_sizes['Train'],dset_sizes['Val'],dset_sizes['Test'])
    return test_acc

In [11]:
model = models.alexnet(pretrained=True)
num_features = model.classifier[6].in_features
features = list(model.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
model.classifier = nn.Sequential(*features) 
model.load_state_dict(torch.load('mytraining_hf_vf.pt'))
model.eval()
acc = test(model)
print((acc))

<class 'torchvision.models.alexnet.AlexNet'>
2900 725 28
0.8214285714285714


In [12]:
print(store_train_loss)

NameError: name 'store_train_loss' is not defined